# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

counter = 0
for sqrt_value in generator:
    print(sqrt_value)
    counter += sqrt_value

print(f"{counter=}")


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
counter=8.382332347441762


In [6]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [50]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
!pip install dlt[duckdb]

In [60]:
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name="census_pipe",destination = "duckdb", dataset_name="census")
info = pipeline.run(people_1(),table_name="people_append",write_disposition="replace")
print(info)

Pipeline census_pipe load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset census
The duckdb destination used duckdb:////content/census_pipe.duckdb location to store data
Load package 1707379049.3943872 is LOADED and contains no failed jobs


In [61]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
conn.sql('show tables')

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_append       │
└─────────────────────┘

In [62]:
conn.sql('select * from people_append')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707379049.3943872 │ 3JPrd57cD+I/Hg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707379049.3943872 │ g1ugtPOg+olIVQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707379049.3943872 │ dwyrjbLB6h1qDw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707379049.3943872 │ 0rBuAq4auY+fRg │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707379049.3943872 │ hvmeGDaZahFlFA │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┴────────────┘

In [63]:
conn.sql('select sum(age) from people_append')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [64]:
info = pipeline.run(people_2(),table_name="people_append",write_disposition="append")
print(info)

Pipeline census_pipe load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset census
The duckdb destination used duckdb:////content/census_pipe.duckdb location to store data
Load package 1707379073.7891927 is LOADED and contains no failed jobs


In [65]:
conn.sql('select * from people_append')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707379049.3943872 │ 3JPrd57cD+I/Hg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707379049.3943872 │ g1ugtPOg+olIVQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707379049.3943872 │ dwyrjbLB6h1qDw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707379049.3943872 │ 0rBuAq4auY+fRg │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707379049.3943872 │ hvmeGDaZahFlFA │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707379073.7891927 │ U1NJNO0cZg0bUw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707379073.7891927 │ jZtBSIBgUmzaUQ │ Job_4      │
│     5 │ 

In [67]:
conn.sql('select sum(age) from people_append')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

In [68]:
conn.sql("drop table _dlt_loads ")
conn.sql("drop table _dlt_pipeline_State ")
conn.sql("drop table _dlt_version ")
conn.sql("drop table people_append ")

In [69]:
conn.close()

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [75]:
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name="census_pipe",destination = "duckdb", dataset_name="census")
info = pipeline.run(people_1(),table_name="people",write_disposition="merge",primary_key = 'id')
print(info)

Pipeline census_pipe load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset census
The duckdb destination used duckdb:////content/census_pipe.duckdb location to store data
Load package 1707379148.2750626 is LOADED and contains no failed jobs


In [76]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
conn.sql('show tables')

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [77]:
conn.sql('select * from people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     2 │ Person_2 │    27 │ City_A  │ 1707379148.2750626 │ SYNlKsmp+QTIsQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707379148.2750626 │ cnBVhsri2Poo0w │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707379148.2750626 │ Qa9LgExQpKzp7w │ NULL       │
│     1 │ Person_1 │    26 │ City_A  │ 1707379148.2750626 │ TrlxEj7PB+89Ng │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707379148.2750626 │ Sx7od/AoSdG8cA │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┴────────────┘

In [78]:
conn.sql('select sum(age),avg(age) from people')

┌──────────┬──────────┐
│ sum(age) │ avg(age) │
│  int128  │  double  │
├──────────┼──────────┤
│      140 │     28.0 │
└──────────┴──────────┘

In [79]:
info = pipeline.run(people_2(),table_name="people",write_disposition="merge",primary_key='id')
print(info)

Pipeline census_pipe load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset census
The duckdb destination used duckdb:////content/census_pipe.duckdb location to store data
Load package 1707379158.9989774 is LOADED and contains no failed jobs


In [80]:
conn.sql('select * from people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     2 │ Person_2 │    27 │ City_A  │ 1707379148.2750626 │ SYNlKsmp+QTIsQ │ NULL       │
│     1 │ Person_1 │    26 │ City_A  │ 1707379148.2750626 │ TrlxEj7PB+89Ng │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1707379158.9989774 │ 3rr0wmmbA9Cl6w │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1707379158.9989774 │ o92hQBUfcxQTJg │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1707379158.9989774 │ rL92Orw77e09Fg │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1707379158.9989774 │ KOaNV73lSHoLeQ │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707379158.9989774 │ fdt8PN+MEW4j3Q │ Job_6      │
│     8 │ 

In [81]:
conn.sql('select sum(age),avg(age) from people')

┌──────────┬──────────┐
│ sum(age) │ avg(age) │
│  int128  │  double  │
├──────────┼──────────┤
│      266 │    33.25 │
└──────────┴──────────┘

In [82]:
conn.sql("drop table _dlt_loads ")
conn.sql("drop table _dlt_pipeline_State ")
conn.sql("drop table _dlt_version ")
conn.sql("drop table people ")

In [83]:
conn.close()

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX